# En este colab se prueba, sobre un conjunto de 4 conversaciones, el modelo finetuneado con un dataset propio de saludos y ayuda. Para esto se utiliza como fuente 4 archivos de audio

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/MAF-ProyectoNLP/Test-Conversaciones/blob/main/Entrenamiento%20propio%20%2B%20Finentuning%20Saludos%20evaluación%20conversación%20desde%20archivos%20de%20audio.ipynb)


*0*. Se instalan las bibliotecas necesarias

In [ ]:
!pip install datasets
!pip install transformers
!pip install SpeechRecognition pydub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.2/224.2 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.3/269.3 kB 20.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 43.2 MB/s eta 0:00

*1*. Se importan las dependencias

In [ ]:
import torch.nn as nn
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import datasets
import numpy as np
import random
import speech_recognition as sr

import glob


from transformers import  RobertaForSequenceClassification, RobertaConfig, RobertaTokenizer


from sklearn.metrics import accuracy_score

from pydub import AudioSegment
from pydub.silence import split_on_silence
from google.colab import files

from pydrive.auth import GoogleAuth
from google.colab import drive
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [ ]:
torch.cuda.empty_cache()
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

*2*. Bajamos el modelo fintuneado con frases de saludo y ayuda. Tambien definimos el tipo de modelo.

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

!mkdir "kfold"
file_id = '1ZlFvCmncexUCWTeK_Ellr9V-NjOLj4m1' 
download = drive.CreateFile({'id': file_id})
download.GetContentFile('kfold/config.json')
file_id ='1-4BA1rCp5HhShofsbA9-cOLDBD6Z9XLl'
download = drive.CreateFile({'id': file_id})
download.GetContentFile('kfold/pytorch_model.bin')

In [ ]:
MODEL_TYPE = "PlanTL-GOB-ES/roberta-large-bne-te"

In [ ]:
MAX_LEN=256

*3*. TestDataset and CompDataset son las clases usadas para la tokenización de la frases, CompDataset se utiliza para entrenar (devuelve también el label), y TestDataset es utilizada para tokenizar las frases para validación

In [ ]:
class TestDataset(Dataset):

    def __init__(self, df):
        self.df_data = df

    def __getitem__(self, index):
        sentence1 = self.df_data.loc[index, 'premise']
        sentence2 = self.df_data.loc[index, 'hypothesis']

        encoded_dict = tokenizer.encode_plus(
                    sentence1, sentence2,      
                    add_special_tokens = True,  
                    max_length = MAX_LEN,           
                    pad_to_max_length = True,
                    return_attention_mask = True,   
                    return_tensors = 'pt',          
                    padding="max_length", 
                    truncation=True
               )
        
        padded_token_list = encoded_dict['input_ids'][0]
        att_mask = encoded_dict['attention_mask'][0]

        sample = (padded_token_list, att_mask)
        return sample

    def __len__(self):
        return len(self.df_data)

In [ ]:
class CompDataset(Dataset):

    def __init__(self, df):
        self.df_data = df

    def __getitem__(self, index):
        sentence1 = self.df_data.loc[index, 'premise']
        sentence2 = self.df_data.loc[index, 'hypothesis']

        encoded_dict = tokenizer.encode_plus(
                    sentence1, sentence2,         
                    add_special_tokens = True, 
                    max_length = MAX_LEN,  
                    pad_to_max_length = True,
                    return_attention_mask = True,
                    return_tensors = 'pt',   
                    padding="max_length", 
                    truncation=True
               )

        padded_token_list = encoded_dict['input_ids'][0]
        att_mask = encoded_dict['attention_mask'][0]
        
        target = torch.tensor(self.df_data.loc[index, 'label'])

        sample = (padded_token_list, att_mask, target)
        return sample

    def __len__(self):
        return len(self.df_data) 

*4*. Se carga el modelo

In [ ]:
tokenizer = RobertaTokenizer.from_pretrained(MODEL_TYPE)

In [ ]:
model_parameters =  RobertaForSequenceClassification.from_pretrained(MODEL_TYPE, num_labels=3)
config = RobertaConfig(
                vocab_size= model_parameters.config.vocab_size,
                hidden_size= model_parameters.config.hidden_size,
                num_hidden_layers= model_parameters.config.num_hidden_layers,
                num_attention_heads=model_parameters.config.num_attention_heads,
                intermediate_size=model_parameters.config.intermediate_size,
                hidden_act=model_parameters.config.hidden_act,
                hidden_dropout_prob=model_parameters.config.hidden_dropout_prob,
                attention_probs_dropout_prob=model_parameters.config.attention_probs_dropout_prob,
                max_position_embeddings=model_parameters.config.max_position_embeddings,
                type_vocab_size=model_parameters.config.type_vocab_size,
                initializer_range=model_parameters.config.initializer_range,
            )

config.num_labels = 3

print(config)

RobertaConfig {
  "attention_probs_dropout_prob": 0.0,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 1024,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.28.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50262
}



In [ ]:
model = RobertaForSequenceClassification(config)
model = RobertaForSequenceClassification.from_pretrained('/content/kfold/')

In [ ]:
print(model.roberta.embeddings.word_embeddings.weight.shape)

torch.Size([50262, 1024])


In [ ]:
model.to(device)

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50262, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-23): 24 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
 

*5* La función getlabel predice el entailment de una frase, que la recibe como una TestDataset 

In [ ]:
def get_label(test_dataloader):
  total = 0

  for _, batch in enumerate(test_dataloader):

    b_input_ids = batch[0].to(device)
    b_input_mask = batch[1].to(device)  

    import os
    os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
    outputs = model(b_input_ids, attention_mask=b_input_mask)
 
    preds = outputs[0].detach().cpu().numpy()

    y_pred = np.argmax(preds, axis=1)
  
  return y_pred

 > predict_matrix calcula una matriz de entailments, indicando en la posición (i,j) el entailment de la hipotesis i sobre la premisa j



In [ ]:
def predict_matrix(hypotesis_to_be_tested,answers_from_operator):
  M = np.zeros([len(hypotesis_to_be_tested),len(answers_from_operator)])
  
  for i in range(len(hypotesis_to_be_tested)):
      for j in range(len(answers_from_operator)):
        dataset = {'premise': [answers_from_operator[j]], 'hypothesis': [hypotesis_to_be_tested[i]]}

        dataloader = torch.utils.data.DataLoader(TestDataset(pd.DataFrame(dataset)), batch_size=1, shuffle=False, num_workers=1)
        M[i,j] = get_label(dataloader)
  return(M)

> Se define una métrica y las estructuras para evaluar las conversaciones, y se crea una matriz de predicciones para poder realizar cálculos sobre la misma

In [ ]:
def init_stats(hypotesis_to_be_tested):
   categories=['Presente','OrdenDistinto','NoPresente']
   global scores
   scores= pd.DataFrame((np.outer(np.zeros(len(hypotesis_to_be_tested)),np.zeros(len(categories)))),columns=categories, index=hypotesis_to_be_tested)

*6*. Se crea la función conversation_analysis, que recibe como parámetros una lista de hipótesis a ser testeadas, y las respuestas brindadas por la operadora. Se utiliza como base la función predict_matrix para indicar para cada hipótesis si se encuentra presente en las respuestas, y en el órden correcto. 



In [ ]:
def conversation_analysis(hypotesis_to_be_tested,answers_from_operator):
  out=predict_matrix(hypotesis_to_be_tested,answers_from_operator)
  hypotesis_ordered= np.ones([len(hypotesis_to_be_tested)])*-1
  for h in range(len(hypotesis_to_be_tested)):
    checker= np.zeros([len(hypotesis_to_be_tested)])
    checker[h]=1
    c=out.T.dot(checker)
    for x in range(len(c)):
      if c[x]==0:
        hypotesis_ordered[h]=x
        break

  ok=[]
  wrong=[]

  for i in range(len(hypotesis_ordered)):
    if hypotesis_ordered[i] ==i:
      ok.append(hypotesis_to_be_tested[i] +" correctamente presente en la comunicación")
    elif hypotesis_ordered[i]<0:
      wrong.append(hypotesis_to_be_tested[i] +" NO PRESENTE en la comunicación")
    else:
      wrong.append(hypotesis_to_be_tested[i] +" presente en la comunicación, pero NO EN EL ORDEN CORRECTO")

  print("** Devolución de la comunicación realizado por la operadora **")
  if len(wrong)>0:
    print("A mejorar:")
  else:
    ok.append("Felicitaciones! Comunicación realizada de forma deseada")
  for w in wrong:
    print(w)

  if len(ok)>0:
    print("Positivo:")
  for o in ok:
    print(o)

*7*. **PRUEBA**

*7.1* La lista de hipótesis a testear

In [ ]:
hypotesis_to_be_tested=["Saludo","Ayuda"]

In [ ]:
init_stats(hypotesis_to_be_tested)

*7.2* Se descargan las conversaciones de prueba del repositorio en GitHub y para cada una se realiza en análisis

In [ ]:
dirTempChunked="Audio_Chunked/"
!mkdir "Audio_Chunked"

In [ ]:
!git clone "https://github.com/MAF-ProyectoNLP/Test-Conversaciones"

Cloning into 'Test-Conversaciones'...
remote: Enumerating objects: 26, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 26 (delta 12), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (26/26), 951.12 KiB | 932.00 KiB/s, done.


> *chunk_conversation divide la conversacion entre las respuestas de la operadora y las del cliente*



In [ ]:
def chunk_conversation(name,sound):
  chunks = split_on_silence(sound, min_silence_len = 100, silence_thresh = -50, seek_step=100 )

  for i, chunk in enumerate(chunks):
      chunk.export('/content/'+dirTempChunked+name+"_phrase_"+str(i)+".wav", format="wav")

> *recognize_Text recibe la ruta de un archivo .wav y aplica la biblioteca de speech recognizer para reconocer el texto en español*





In [ ]:
def recognize_Text(filename):
  r = sr.Recognizer()
  with sr.AudioFile(filename) as source:
      audio_data = r.record(source)
      text = r.recognize_google(audio_data,language="es-US",show_all=False)
      return text

Para cada archivo en la carpeta se realiza el análisis y se imprime su salida

In [ ]:
conversations = glob.glob("Test-Conversaciones" + '/*.mp3')

for conv in conversations:
  try:
    name=conv.split("/")[1]
    print(name)
    chunk_conversation(name,AudioSegment.from_file(conv))
    answers_from_operator=[]

    answers_from_operator.append(recognize_Text(filename="/content/"+dirTempChunked+name+"_phrase_0.wav"))
    answers_from_operator.append(recognize_Text(filename="/content/"+dirTempChunked+name+"_phrase_2.wav"))
    print("Respuestas detectadas en el audio:")
    print(answers_from_operator)
    print("\n")
    conversation_analysis(hypotesis_to_be_tested,answers_from_operator)
    print("\n\n---------------------------------------------------------")

  except:
    print("error en reconocimiento de voz")

conversacion1.mp3
Respuestas detectadas en el audio:
['Hola', 'en qué lo puedo ayudar el día de hoy']


** Devolución de la comunicación realizado por la operadora **
Positivo:
Saludo correctamente presente en la comunicación
Ayuda correctamente presente en la comunicación
Felicitaciones! Comunicación realizada de forma deseada


---------------------------------------------------------
conversacion3.mp3
Respuestas detectadas en el audio:
['en qué lo puedo ayudar el día de hoy', 'Hola']


** Devolución de la comunicación realizado por la operadora **
A mejorar:
Saludo presente en la comunicación, pero NO EN EL ORDEN CORRECTO
Ayuda presente en la comunicación, pero NO EN EL ORDEN CORRECTO


---------------------------------------------------------
conversacion2.mp3
Respuestas detectadas en el audio:
['Hola', 'qué cansancio']


** Devolución de la comunicación realizado por la operadora **
A mejorar:
Ayuda NO PRESENTE en la comunicación
Positivo:
Saludo correctamente presente en la comun